# Option Chain Exploration with yfinance

This notebook demonstrates how to:
1. Fetch option chain data using yfinance
2. Explore the data structure
3. Visualize option prices, volumes, and open interest
4. Calculate basic Greeks using our Black-Scholes implementation
5. Analyze implied volatility patterns

We'll use a popular ticker like SPY (S&P 500 ETF) or AAPL for this exploration.

In [1]:
# Import necessary packages
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Import our custom pricing functions
import sys
sys.path.append('../src')
from options_desk.pricing.black_scholes import (
    black_scholes_price,
    black_scholes_delta,
    black_scholes_gamma,
    black_scholes_vega,
    black_scholes_theta
)
from options_desk.core.option import OptionType

## 1. Fetch Underlying Stock Data

In [2]:
# Choose a ticker to analyze
TICKER = 'SPY'  # S&P 500 ETF - liquid and active options market
# TICKER = 'AAPL'  # Alternative: Apple stock

# Create ticker object
ticker = yf.Ticker(TICKER)

# Get current stock price
stock_data = ticker.history(period='5d')
current_price = stock_data['Close'].iloc[-1]

print(f"Ticker: {TICKER}")
print(f"Current Price: ${current_price:.2f}")
print(f"\nLast 5 days of price data:")
stock_data[['Close', 'Volume']].tail()

Ticker: SPY
Current Price: $670.97

Last 5 days of price data:


,Close,Volume
Date,,
2025-11-03 00:00:00-05:00,683.340027,57315000
2025-11-04 00:00:00-05:00,675.239990,78427000
2025-11-05 00:00:00-05:00,677.580017,74402400
2025-11-06 00:00:00-05:00,670.309998,85035300
2025-11-07 00:00:00-05:00,670.969971,100516700


## 2. Explore Available Option Expiration Dates

In [ ]:
# Get available expiration dates
expiration_dates = ticker.options

print(f"Available expiration dates ({len(expiration_dates)} total):")
print("\nFirst 10 expiration dates:")
for i, date in enumerate(expiration_dates[:10], 1):
    exp_date = pd.to_datetime(date)
    days_to_exp = (exp_date - pd.Timestamp.now()).days
    print(f"{i:2d}. {date} ({days_to_exp:3d} days to expiry)")

## 3. Fetch Option Chain for a Specific Expiration

In [ ]:
# Select an expiration date (typically 30-60 days out is most liquid)
# We'll use the 3rd expiration date as it's often around 30-45 days
expiry_date = expiration_dates[2] if len(expiration_dates) >= 3 else expiration_dates[0]

# Fetch option chain
opt_chain = ticker.option_chain(expiry_date)

# Separate calls and puts
calls = opt_chain.calls
puts = opt_chain.puts

print(f"\n{'='*70}")
print(f"Option Chain for {TICKER} expiring on {expiry_date}")
print(f"{'='*70}")
print(f"\nNumber of call options: {len(calls)}")
print(f"Number of put options: {len(puts)}")

# Calculate days to expiration
expiry_datetime = pd.to_datetime(expiry_date)
days_to_expiry = (expiry_datetime - pd.Timestamp.now()).days
time_to_expiry = days_to_expiry / 365.25

print(f"\nDays to expiration: {days_to_expiry}")
print(f"Time to expiry (years): {time_to_expiry:.4f}")

## 4. Examine Option Chain Structure

In [ ]:
# Display first few call options
print("\nSample CALL options:")
print("Columns:", calls.columns.tolist())
print("\nFirst 10 call strikes around ATM:")
atm_calls = calls[calls['strike'].between(current_price * 0.95, current_price * 1.05)]
display(atm_calls[['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']].head(10))

In [ ]:
# Display first few put options
print("\nSample PUT options:")
print("\nFirst 10 put strikes around ATM:")
atm_puts = puts[puts['strike'].between(current_price * 0.95, current_price * 1.05)]
display(atm_puts[['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']].head(10))

## 5. Visualize Option Prices Across Strikes

In [ ]:
# Plot option prices across strikes
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Call prices
axes[0].plot(calls['strike'], calls['lastPrice'], 'o-', color='green', alpha=0.6, label='Last Price')
axes[0].plot(calls['strike'], (calls['bid'] + calls['ask']) / 2, 's-', color='darkgreen', alpha=0.4, label='Mid Price')
axes[0].axvline(current_price, color='red', linestyle='--', linewidth=2, label=f'Spot: ${current_price:.2f}')
axes[0].set_xlabel('Strike Price', fontsize=12)
axes[0].set_ylabel('Option Price ($)', fontsize=12)
axes[0].set_title(f'CALL Option Prices - {TICKER} (Expiry: {expiry_date})', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Put prices
axes[1].plot(puts['strike'], puts['lastPrice'], 'o-', color='red', alpha=0.6, label='Last Price')
axes[1].plot(puts['strike'], (puts['bid'] + puts['ask']) / 2, 's-', color='darkred', alpha=0.4, label='Mid Price')
axes[1].axvline(current_price, color='green', linestyle='--', linewidth=2, label=f'Spot: ${current_price:.2f}')
axes[1].set_xlabel('Strike Price', fontsize=12)
axes[1].set_ylabel('Option Price ($)', fontsize=12)
axes[1].set_title(f'PUT Option Prices - {TICKER} (Expiry: {expiry_date})', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Analyze Trading Volume and Open Interest

In [ ]:
# Plot volume and open interest
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Call volume
axes[0, 0].bar(calls['strike'], calls['volume'], color='green', alpha=0.6, width=1)
axes[0, 0].axvline(current_price, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Strike Price')
axes[0, 0].set_ylabel('Volume')
axes[0, 0].set_title('CALL Volume by Strike', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Put volume
axes[0, 1].bar(puts['strike'], puts['volume'], color='red', alpha=0.6, width=1)
axes[0, 1].axvline(current_price, color='green', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Strike Price')
axes[0, 1].set_ylabel('Volume')
axes[0, 1].set_title('PUT Volume by Strike', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Call open interest
axes[1, 0].bar(calls['strike'], calls['openInterest'], color='green', alpha=0.6, width=1)
axes[1, 0].axvline(current_price, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Strike Price')
axes[1, 0].set_ylabel('Open Interest')
axes[1, 0].set_title('CALL Open Interest by Strike', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Put open interest
axes[1, 1].bar(puts['strike'], puts['openInterest'], color='red', alpha=0.6, width=1)
axes[1, 1].axvline(current_price, color='green', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Strike Price')
axes[1, 1].set_ylabel('Open Interest')
axes[1, 1].set_title('PUT Open Interest by Strike', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Implied Volatility Analysis

In [ ]:
# Plot implied volatility smile/skew
fig, ax = plt.subplots(figsize=(14, 6))

# Calculate moneyness (K/S)
calls['moneyness'] = calls['strike'] / current_price
puts['moneyness'] = puts['strike'] / current_price

# Plot IV smile
ax.plot(calls['moneyness'], calls['impliedVolatility'] * 100, 'o-', color='green', 
        alpha=0.6, label='Calls', markersize=4)
ax.plot(puts['moneyness'], puts['impliedVolatility'] * 100, 'o-', color='red', 
        alpha=0.6, label='Puts', markersize=4)
ax.axvline(1.0, color='black', linestyle='--', linewidth=2, label='ATM')

ax.set_xlabel('Moneyness (Strike / Spot)', fontsize=12)
ax.set_ylabel('Implied Volatility (%)', fontsize=12)
ax.set_title(f'Implied Volatility Smile/Skew - {TICKER} (Expiry: {expiry_date})', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print some statistics
print("\nImplied Volatility Statistics:")
print(f"\nCALLS:")
print(f"  Min IV: {calls['impliedVolatility'].min()*100:.2f}%")
print(f"  Max IV: {calls['impliedVolatility'].max()*100:.2f}%")
print(f"  Mean IV: {calls['impliedVolatility'].mean()*100:.2f}%")
print(f"  ATM IV: {calls.iloc[(calls['strike'] - current_price).abs().argsort()[:1]]['impliedVolatility'].values[0]*100:.2f}%")

print(f"\nPUTS:")
print(f"  Min IV: {puts['impliedVolatility'].min()*100:.2f}%")
print(f"  Max IV: {puts['impliedVolatility'].max()*100:.2f}%")
print(f"  Mean IV: {puts['impliedVolatility'].mean()*100:.2f}%")
print(f"  ATM IV: {puts.iloc[(puts['strike'] - current_price).abs().argsort()[:1]]['impliedVolatility'].values[0]*100:.2f}%")

## 8. Calculate Greeks Using Our Black-Scholes Implementation

In [ ]:
# Select ATM options for detailed analysis
atm_strike = calls.iloc[(calls['strike'] - current_price).abs().argsort()[:1]]['strike'].values[0]
atm_call = calls[calls['strike'] == atm_strike].iloc[0]
atm_put = puts[puts['strike'] == atm_strike].iloc[0]

# Use implied volatility from market
iv_call = atm_call['impliedVolatility']
iv_put = atm_put['impliedVolatility']
avg_iv = (iv_call + iv_put) / 2

# Risk-free rate (approximate - you can use Treasury rates)
risk_free_rate = 0.045  # 4.5% - adjust based on current rates

print(f"\n{'='*70}")
print(f"ATM Analysis for Strike: ${atm_strike:.2f}")
print(f"{'='*70}")
print(f"\nCurrent Spot Price: ${current_price:.2f}")
print(f"Time to Expiry: {days_to_expiry} days ({time_to_expiry:.4f} years)")
print(f"Average Implied Volatility: {avg_iv*100:.2f}%")
print(f"Risk-free Rate: {risk_free_rate*100:.2f}%")

# Calculate theoretical prices and Greeks for CALL
print(f"\n{'='*35} CALL {'='*35}")
call_price_theo = black_scholes_price(
    current_price, atm_strike, time_to_expiry, risk_free_rate, avg_iv, OptionType.CALL
)
call_delta = black_scholes_delta(
    current_price, atm_strike, time_to_expiry, risk_free_rate, avg_iv, OptionType.CALL
)
call_gamma = black_scholes_gamma(
    current_price, atm_strike, time_to_expiry, risk_free_rate, avg_iv
)
call_vega = black_scholes_vega(
    current_price, atm_strike, time_to_expiry, risk_free_rate, avg_iv
)
call_theta = black_scholes_theta(
    current_price, atm_strike, time_to_expiry, risk_free_rate, avg_iv, OptionType.CALL
)

print(f"Market Price: ${atm_call['lastPrice']:.2f} (bid: ${atm_call['bid']:.2f}, ask: ${atm_call['ask']:.2f})")
print(f"Theoretical Price: ${call_price_theo:.2f}")
print(f"\nGreeks:")
print(f"  Delta: {call_delta:.4f} (price change per $1 move in underlying)")
print(f"  Gamma: {call_gamma:.4f} (delta change per $1 move in underlying)")
print(f"  Vega: {call_vega:.4f} (price change per 1% change in IV)")
print(f"  Theta: {call_theta:.4f} (price change per day)")

# Calculate theoretical prices and Greeks for PUT
print(f"\n{'='*35} PUT {'='*35}")
put_price_theo = black_scholes_price(
    current_price, atm_strike, time_to_expiry, risk_free_rate, avg_iv, OptionType.PUT
)
put_delta = black_scholes_delta(
    current_price, atm_strike, time_to_expiry, risk_free_rate, avg_iv, OptionType.PUT
)
put_vega = black_scholes_vega(
    current_price, atm_strike, time_to_expiry, risk_free_rate, avg_iv
)
put_theta = black_scholes_theta(
    current_price, atm_strike, time_to_expiry, risk_free_rate, avg_iv, OptionType.PUT
)

print(f"Market Price: ${atm_put['lastPrice']:.2f} (bid: ${atm_put['bid']:.2f}, ask: ${atm_put['ask']:.2f})")
print(f"Theoretical Price: ${put_price_theo:.2f}")
print(f"\nGreeks:")
print(f"  Delta: {put_delta:.4f} (price change per $1 move in underlying)")
print(f"  Gamma: {call_gamma:.4f} (same as call - gamma is identical for calls and puts)")
print(f"  Vega: {put_vega:.4f} (price change per 1% change in IV)")
print(f"  Theta: {put_theta:.4f} (price change per day)")

## 9. Greeks Profile Across Strikes

In [ ]:
# Calculate Greeks for a range of strikes
strike_range = calls['strike'].values

# Initialize arrays for Greeks
deltas_call = []
deltas_put = []
gammas = []
vegas = []

for strike in strike_range:
    # Use implied volatility from the options chain
    iv = calls[calls['strike'] == strike]['impliedVolatility'].values[0] if len(calls[calls['strike'] == strike]) > 0 else avg_iv
    
    delta_c = black_scholes_delta(current_price, strike, time_to_expiry, risk_free_rate, iv, OptionType.CALL)
    delta_p = black_scholes_delta(current_price, strike, time_to_expiry, risk_free_rate, iv, OptionType.PUT)
    gamma = black_scholes_gamma(current_price, strike, time_to_expiry, risk_free_rate, iv)
    vega = black_scholes_vega(current_price, strike, time_to_expiry, risk_free_rate, iv)
    
    deltas_call.append(delta_c)
    deltas_put.append(delta_p)
    gammas.append(gamma)
    vegas.append(vega)

# Plot Greeks
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Delta
axes[0, 0].plot(strike_range, deltas_call, 'g-', linewidth=2, label='Call Delta')
axes[0, 0].plot(strike_range, deltas_put, 'r-', linewidth=2, label='Put Delta')
axes[0, 0].axvline(current_price, color='black', linestyle='--', linewidth=1.5)
axes[0, 0].axhline(0, color='gray', linestyle='-', linewidth=0.5)
axes[0, 0].set_xlabel('Strike Price')
axes[0, 0].set_ylabel('Delta')
axes[0, 0].set_title('Delta across Strikes', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Gamma
axes[0, 1].plot(strike_range, gammas, 'b-', linewidth=2)
axes[0, 1].axvline(current_price, color='black', linestyle='--', linewidth=1.5, label='Spot')
axes[0, 1].set_xlabel('Strike Price')
axes[0, 1].set_ylabel('Gamma')
axes[0, 1].set_title('Gamma across Strikes', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Vega
axes[1, 0].plot(strike_range, vegas, 'purple', linewidth=2)
axes[1, 0].axvline(current_price, color='black', linestyle='--', linewidth=1.5, label='Spot')
axes[1, 0].set_xlabel('Strike Price')
axes[1, 0].set_ylabel('Vega')
axes[1, 0].set_title('Vega across Strikes', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Combined view - normalize to see relative shapes
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
deltas_call_norm = scaler.fit_transform(np.array(deltas_call).reshape(-1, 1)).flatten()
gammas_norm = scaler.fit_transform(np.array(gammas).reshape(-1, 1)).flatten()
vegas_norm = scaler.fit_transform(np.array(vegas).reshape(-1, 1)).flatten()

axes[1, 1].plot(strike_range, deltas_call_norm, 'g-', linewidth=2, label='Delta (normalized)')
axes[1, 1].plot(strike_range, gammas_norm, 'b-', linewidth=2, label='Gamma (normalized)')
axes[1, 1].plot(strike_range, vegas_norm, color='purple', linewidth=2, label='Vega (normalized)')
axes[1, 1].axvline(current_price, color='black', linestyle='--', linewidth=1.5)
axes[1, 1].set_xlabel('Strike Price')
axes[1, 1].set_ylabel('Normalized Value')
axes[1, 1].set_title('Greeks Comparison (Normalized)', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Put-Call Parity Check

Put-Call Parity states that:
$$C - P = S - K \cdot e^{-r \cdot T}$$

Where:
- C = Call price
- P = Put price  
- S = Spot price
- K = Strike price
- r = Risk-free rate
- T = Time to expiration

Violations of this relationship can indicate arbitrage opportunities (or more likely, bid-ask spread effects).

In [ ]:
# Merge calls and puts on strike
parity_df = pd.merge(
    calls[['strike', 'lastPrice', 'bid', 'ask']],
    puts[['strike', 'lastPrice', 'bid', 'ask']],
    on='strike',
    suffixes=('_call', '_put')
)

# Calculate put-call parity components
parity_df['call_minus_put'] = parity_df['lastPrice_call'] - parity_df['lastPrice_put']
parity_df['spot_minus_pv_strike'] = current_price - parity_df['strike'] * np.exp(-risk_free_rate * time_to_expiry)
parity_df['parity_violation'] = parity_df['call_minus_put'] - parity_df['spot_minus_pv_strike']

# Plot put-call parity
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(parity_df['spot_minus_pv_strike'], parity_df['call_minus_put'], alpha=0.6)
# Add perfect parity line
min_val = min(parity_df['spot_minus_pv_strike'].min(), parity_df['call_minus_put'].min())
max_val = max(parity_df['spot_minus_pv_strike'].max(), parity_df['call_minus_put'].max())
axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Parity')
axes[0].set_xlabel('S - PV(K) = Theoretical')
axes[0].set_ylabel('C - P = Market')
axes[0].set_title('Put-Call Parity: Market vs Theoretical', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Parity violations
axes[1].bar(parity_df['strike'], parity_df['parity_violation'], alpha=0.6, width=1)
axes[1].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1].axvline(current_price, color='green', linestyle='--', linewidth=2, label='Spot')
axes[1].set_xlabel('Strike Price')
axes[1].set_ylabel('Parity Violation ($)')
axes[1].set_title('Put-Call Parity Violations by Strike', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPut-Call Parity Statistics:")
print(f"Mean Violation: ${parity_df['parity_violation'].mean():.4f}")
print(f"Std Violation: ${parity_df['parity_violation'].std():.4f}")
print(f"Max Violation: ${parity_df['parity_violation'].max():.4f}")
print(f"Min Violation: ${parity_df['parity_violation'].min():.4f}")

## 11. Summary and Next Steps

### What we've learned:
1. How to fetch real-time option chain data using yfinance
2. Structure of option market data (strikes, prices, volume, open interest)
3. Implied volatility smile/skew patterns
4. Greeks calculation using our Black-Scholes implementation
5. Put-call parity relationships

### Ideas for `src` module structure based on this exploration:

```
src/options_desk/
├── data/
│   ├── fetchers.py          # yfinance wrapper, data fetching logic
│   ├── parsers.py           # Parse option chain to OptionContract objects
│   └── cache.py             # Optional: cache market data
├── surface/
│   ├── iv_surface.py        # Build volatility surfaces from market data
│   ├── interpolation.py     # Interpolate IV across strikes/expiries
│   └── arbitrage.py         # Check for arbitrage opportunities
├── analytics/
│   ├── greeks.py            # Batch Greeks calculation for portfolios
│   ├── parity.py            # Put-call parity checks
│   └── moneyness.py         # Moneyness calculations and utilities
└── visualization/
    ├── plots.py             # Reusable plotting functions
    └── dashboards.py        # Optional: interactive dashboards
```

### Next notebooks to create:
1. **02_implied_volatility.ipynb** - Deep dive into IV calculation and surface construction
2. **03_greeks_analysis.ipynb** - Portfolio Greeks and risk management
3. **04_delta_hedging.ipynb** - Simulate delta hedging strategies
4. **05_backtest.ipynb** - Backtest option strategies on historical data

In [ ]:
# Quick summary statistics
print(f"\n{'='*70}")
print(f"EXPLORATION SUMMARY - {TICKER}")
print(f"{'='*70}")
print(f"\nData Quality:")
print(f"  Total call options analyzed: {len(calls)}")
print(f"  Total put options analyzed: {len(puts)}")
print(f"  Options with volume > 0: {len(calls[calls['volume'] > 0])} calls, {len(puts[puts['volume'] > 0])} puts")
print(f"  Options with OI > 0: {len(calls[calls['openInterest'] > 0])} calls, {len(puts[puts['openInterest'] > 0])} puts")

print(f"\nMarket Characteristics:")
print(f"  Current spot: ${current_price:.2f}")
print(f"  ATM strike: ${atm_strike:.2f}")
print(f"  ATM call IV: {iv_call*100:.2f}%")
print(f"  ATM put IV: {iv_put*100:.2f}%")
print(f"  Days to expiration: {days_to_expiry}")

print(f"\n\nGreat job! You now have a solid foundation for understanding option market data.")
print(f"Try changing the TICKER or expiry_date to explore different options.")